In [15]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
from shapely.geometry import box, mapping
import shapely.geometry
import rasterio.features
import contextily as cx  # optional for basemap
import numpy as np
import yaml
import os

# Load configuration
with open('../../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

def resolve_path(relative_path):
    return os.path.join(config['base_path'], relative_path)

In [16]:
# === Load Data ===
cpis = gpd.read_file(resolve_path(config['SSA_Combined_CPIS_All_shp_path']))
dams = gpd.read_file(resolve_path(config['AridAfrica_Barriers_shp_path']))
ca = gpd.read_file(resolve_path(config['No_Crop_Vectorized_Command_Area_shp_path']))
country_boundaries = gpd.read_file(resolve_path(config['Africa_boundaries_shp_path']))
irrig_raster_path = resolve_path(config['Africa_AEI_2000_asc_path'])

# Focus on GDW_ID 407
example_dam = dams[dams["GDW_ID"] == 407]
ca_407 = ca[ca["GDW_ID"] == 407]

# Reproject all to Web Mercator (EPSG:3857) for plotting with basemap
ca_407 = ca_407.to_crs(epsg=3857)
cpis = cpis.to_crs(epsg=3857)
example_dam = example_dam.to_crs(epsg=3857)

# Crop raster to extent
buffer_dist = 100000  # 100km
bbox = ca_407.geometry.buffer(buffer_dist).total_bounds
bbox_geom = box(*bbox)
bbox_gdf = gpd.GeoDataFrame({"geometry": [bbox_geom]}, crs=ca_407.crs)
cpis_clip = gpd.overlay(cpis, bbox_gdf, how="intersection")


# === Load and Clip Raster ===
with rasterio.open(irrig_raster_path) as src:
    irrig_crs = "EPSG:4326"  # manually assign CRS
    bbox_geom_json = [mapping(bbox_geom)]  # Convert to GeoJSON format
    out_image, out_transform = mask(src, bbox_geom_json, crop=True)
    extent = [
        out_transform[2],
        out_transform[2] + out_transform[0] * out_image.shape[2],
        out_transform[5] + out_transform[4] * out_image.shape[1],
        out_transform[5],
    ]
    

# Mask out non-irrigated pixels
masked = np.ma.masked_where(out_image[0] <= 0, out_image[0])

# === Plot ===
fig, ax = plt.subplots(figsize=(12, 12))

# Raster: Irrigated area (stretched to better show variation)
img = ax.imshow(masked, cmap="viridis", alpha=0.7, extent=extent, origin='upper', vmin=1, vmax=np.percentile(masked.compressed(), 95))

# Command area outline
ca_407.boundary.plot(ax=ax, color='red', linewidth=2, label="Command Area")

# CPIS polygons
cpis_clip.plot(ax=ax, facecolor='blue', edgecolor='black', linewidth=0.5, alpha=0.7, label="CPIS")

# Dam location
example_dam.plot(ax=ax, color='yellow', marker='*', markersize=150, label="Dam")

# Set zoom to buffered area
ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])

# Add basemap under everything
cx.add_basemap(ax, crs=ca_407.crs, source=cx.providers.OpenStreetMap.Mapnik)

# Colorbar for irrigated area
cbar = plt.colorbar(img, ax=ax, shrink=0.5, label="Irrigated Area (sq meters)")

# Final polish
ax.set_title("Dam 407: Command Area, Irrigated Land, and CPIS Overlay")
ax.axis('off')
ax.legend()
plt.tight_layout()
plt.show()

ValueError: Input shapes do not overlap raster.